<a href="https://colab.research.google.com/github/nuryte/CSE498-AML-EDWARD-JEFFS-MIA/blob/main/testsarsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
import gym
from gym import spaces
#import pygame
import numpy as np
from panel import state

class SawWorldEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}

    def __init__(self, render_mode=None):
        self.window_size = 512  # The size of the PyGame window

        # Observations are dictionaries with the agent's and the target's location.
        # Each location is encoded as an element of {0, ..., `size`}^2, i.e. MultiDiscrete([size, size]).
        
        self.actions = np.array([ [1,1], [1,0], [1,-1],[0,1], [0,0],[0,-1]])
        self.max_values = np.array([15,6,3,3,len(self.actions)])
        self.target_location = np.array([self.max_values[0]-self.max_values[2],self.max_values[1], self.max_values[2],self.max_values[3]]) #dmax= 100, thetamax = 90, ddmax = 20, dthetamax = 20

        self.angle_scale = .5
        self.observation_space = np.array([self.max_values[2],self.max_values[1], self.max_values[2],self.max_values[3]])#d, theta, dd, dtheta, actins
        self.states = self.max_values.copy()
        self.states[1:4] = self.states[1:4]*2+1
        
        

   

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode

        """
        If human-rendering is used, `self.window` will be a reference
        to the window that we draw to. `self.clock` will be a clock that is used
        to ensure that the environment is rendered at the correct framerate in
        human-mode. They will remain `None` until human-mode is used for the
        first time.
        """
        self.window = None
        self.clock = None


        
        
    def _get_obs(self):
        return self.observation_space
    
    def _get_info(self):
        return -1#{"distance": np.linalg.norm(self._agent_location - self._target_location, ord=1)}
    
    def reset(self, seed=None, options=None):
        #self.close()
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Choose the agent's location uniformly at random
        
        self.observation_space = np.array([self.max_values[0],self.max_values[1], self.max_values[2],self.max_values[3]])#d, theta, dd, dtheta, actins

        # We will sample the target's location randomly until it does not coincide with the agent's location

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info
    
    def step(self, action):
        terminated = False
        reward = 0
        
        self.observation_space = self.move_saw(self.observation_space, action)

        terminated, reward = self.find_terminated(self.observation_space)



        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, False, info
    
    def render(self):
        if self.render_mode == "rgb_array":
            return self._render_frame()

    def _render_frame(self):
        pass
        
    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()
    
    def move_saw(self, state, action):

        if np.array_equal(state[0:2], self.target_location.astype(int)[0:2]):
          state[4] = 0
          return state

        dd =  int(state[2] + self.angle_scale*(state[1]-self.max_values[1])* action[0]) #dd
        dtheta = state[3] + action[1] #dtheta


        if dd >= self.states[2]-1:
            state[2] = self.states[2]-1
        elif dd < 0:
            state[2] = 0
        else:
            state[2] = dd

        if dtheta >= self.states[3]-1:
            state[3] = self.states[3]-1
        elif dtheta < 0:
            state[3] = 0
        else:
            state[3] = dtheta

        state[0] = state[0] + (state[2] - self.max_values[2])
        state[1] = state[1] + (state[3] - self.max_values[3])
        
        if state[0] >= self.states[0]-1:
          state[0] = self.states[0]-1
        elif state[0] < 0:
          state[0] = 0
          
        if state[1] >= self.states[1]-1:
          state[1] = self.states[1]-1
        elif state[1] < 0:
          state[1] = 0
        state[4] = -1
        
        if np.array_equal(state[0:2], self.target_location.astype(int)[0:2]):
          state[4] = 0
          
        return state

    def find_terminated(self, state):
        reward = -1
        terminated = False
        
        #if state[0] < 0 or state[0] >= self.states[0]:
            #terminated = True
        #elif state[1] < 0 or state[1] >= self.states[1]:
            #terminated = True
        if np.array_equal(state[0:4], self.target_location.astype(int)):
            terminated = True
            print("found exit")
            reward = 0

        return terminated, reward

    def all_next_states(self):
      all_states_dims = np.array([self.states[0],self.states[1],self.states[2],self.states[3],self.states[4],5])
      all_states = np.zeros(all_states_dims)
      print(all_states.shape)

      for i in range(self.states[0]):
        for j in range(self.states[1]):
            for k in range(self.states[2]):
                for l in range(self.states[3]):
                  state = np.array([i,j,k,l,-1]).astype(int)
                  
                  for m in range(0,self.max_values[4]):
                  
                    stateNext = self.move_saw(state.copy(), self.actions[m])
                    if stateNext[4] == 0:
                      print("hype: ", state, self.actions[m])
                    all_states[i][j][k][l][m] = stateNext.copy()
      return all_states.astype(int)
                      


In [131]:


def evaluate(policy, values, all_states ,env, gamma):
    
    
    nextValues = np.zeros(np.shape(values))
    #print("evaluate")
        
    for i in range(env.states[0]):
        for j in range(env.states[1]):
            for k in range(env.states[2]):
                for l in range(env.states[3]):
                  
                    stateval = 0
                    
                    for m in range(0,env.max_values[4]):
                        
                        stateNext = all_states[i][j][k][l][m]
                        
                        stateval += policy[i][j][k][l][m] * (stateNext[4] + gamma * values[stateNext[0]][stateNext[1]][stateNext[2]][stateNext[3]])
                        
                    
                    nextValues[i][j][k][l] = stateval
                        
    return nextValues
        

        
def iterate(policy, values,all_states, env,gamma):
    
    
    
    nextpolicy = np.zeros(np.shape(policy))
    print("iterate")
    for i in range(env.states[0]):
        #print(i)
        for j in range(env.states[1]):
            for k in range(env.states[2]):
                for l in range(env.states[3]):
                    stateval = []
                    for m in range(0,env.max_values[4]):
                        
                        stateNext = all_states[i][j][k][l][m]
                        nextval = policy[i][j][k][l][m] * (stateNext[4] + gamma * values[stateNext[0]][stateNext[1]][stateNext[2]][stateNext[3]])
                        stateval.append(nextval)
                    
                    maxval = stateval[np.argmax(stateval)]
                    count = (stateval == maxval).sum()
                    for m in range(0,env.max_values[4]):
                        nextpolicy[i][j][k][l][m] = 1/count
    return nextpolicy
                        

def knownGridworld(next_states):
    env = SawWorldEnv(render_mode = "rgb_array")
    gamma = 1
    #next_states = env.all_next_states().astype(int)
    
    states = env.states

    nextvals = np.zeros(states[0:4])

    policy = np.ones(states) * 1/states[4]
    print(policy.size)
    #nextvals = evaluate(policy, values, env)
    
    for y in range (10):
        #nextvals = np.zeros(states[0:4])
        for x in range(100000):
            values = nextvals.copy()
            #print(nextvals)
            nextvals = evaluate(policy, values, next_states,env,gamma)
            delta = np.sum(np.abs(nextvals - values))
            if x%50 == 0:
              print("eval: ",delta)
            if delta <.001:
                break
        
        newpolicy = iterate(policy, nextvals,next_states, env,gamma)
        delpol = np.sum(np.abs(newpolicy - policy))
        print("iter: ",delpol)
        if delpol < .1:
            break
        policy = newpolicy
    
    env.close()
    return newpolicy

def test():
    env = SawWorldEnv(render_mode = "rgb_array")
    all_states = env.all_next_states()
    print(all_states.shape)
    states = env.states

    values = np.zeros(states[0:4])

    policy = np.ones(states) * 1/6
    print(policy.size)
    nextvals = evaluate(policy, values, all_states, env)
    return
    newpolicy = iterate(policy, nextvals, all_states, env)
#newpol = knownGridworld()
#test()

In [125]:
env = SawWorldEnv(render_mode = "rgb_array")
all_states = env.all_next_states().astype(int)
print(all_states.shape)
    

(15, 13, 7, 7, 6, 5)
hype:  [ 9  3  6  5 -1] [0 1]
hype:  [ 9  3  6  6 -1] [0 1]
hype:  [ 9  3  6  6 -1] [0 0]
hype:  [ 9  4  6  4 -1] [0 1]
hype:  [ 9  4  6  5 -1] [0 0]
hype:  [ 9  4  6  6 -1] [ 0 -1]
hype:  [ 9  5  6  3 -1] [0 1]
hype:  [ 9  5  6  4 -1] [0 0]
hype:  [ 9  5  6  5 -1] [ 0 -1]
hype:  [ 9  6  6  2 -1] [1 1]
hype:  [ 9  6  6  2 -1] [0 1]
hype:  [ 9  6  6  3 -1] [1 0]
hype:  [ 9  6  6  3 -1] [0 0]
hype:  [ 9  6  6  4 -1] [ 1 -1]
hype:  [ 9  6  6  4 -1] [ 0 -1]
hype:  [ 9  7  6  1 -1] [1 1]
hype:  [ 9  7  6  1 -1] [0 1]
hype:  [ 9  7  6  2 -1] [1 0]
hype:  [ 9  7  6  2 -1] [0 0]
hype:  [ 9  7  6  3 -1] [ 1 -1]
hype:  [ 9  7  6  3 -1] [ 0 -1]
hype:  [ 9  8  5  0 -1] [1 1]
hype:  [ 9  8  5  1 -1] [1 0]
hype:  [ 9  8  5  2 -1] [ 1 -1]
hype:  [ 9  8  6  0 -1] [1 1]
hype:  [ 9  8  6  0 -1] [0 1]
hype:  [ 9  8  6  1 -1] [1 0]
hype:  [ 9  8  6  1 -1] [0 0]
hype:  [ 9  8  6  2 -1] [ 1 -1]
hype:  [ 9  8  6  2 -1] [ 0 -1]
hype:  [ 9  9  5  0 -1] [1 0]
hype:  [ 9  9  5  0 -1] [ 1 -1]

In [132]:
#states = env.states

#values = np.zeros(states[0:4])

#policy = np.ones(states) * 1/6
#print(policy.size)
#ex = all_states[0][0][0][0][0].astype(int)
#print(ex[0])
#print(values[ex[0]][ex[1]][ex[2]][ex[3]])
#nextvals = evaluate(policy, values, all_states.astype(int), env)

#newpolicy = iterate(policy, nextvals, all_states.astype(int), env)
knownGridworld(all_states)

57330
eval:  9466.0
eval:  9364.32823641224
eval:  9335.018191707877
eval:  9305.799681346823
eval:  9276.67261763738
eval:  9247.636721132132
eval:  9218.691706697631
eval:  9189.837289881172
eval:  9161.073187112965
eval:  9132.399115711276
eval:  9103.81479387873
eval:  9075.319940699907
eval:  9046.91427613883
eval:  9018.597521036081
eval:  8990.369397105656
eval:  8962.22962693301
eval:  8934.177933971398
eval:  8906.214042540503
eval:  8878.337677822583
eval:  8850.548565859044
eval:  8822.84643355
eval:  8795.231008650237
eval:  8767.702019765844
eval:  8740.25919635341
eval:  8712.902268715643
eval:  8685.63096799929
eval:  8658.44502619357
eval:  8631.344176125343
eval:  8604.328151458032
eval:  8577.396686689946
eval:  8550.549517148651
eval:  8523.78637899116
eval:  8497.10700919989
eval:  8470.511145579847
eval:  8443.998526759397
eval:  8417.568892182326
eval:  8391.221982108224
eval:  8364.957537610619
eval:  8338.775300572845
eval:  8312.675013686305
eval:  8286.6564204

KeyboardInterrupt: ignored